In [1]:
import numpy as np
import cv2
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import os

Using TensorFlow backend.
/home/sandesh/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sandesh/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sandesh/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sandesh/anaconda3/lib/python3.7/site-packages/t

In [2]:
current_dir = os.getcwd()
model = load_model(current_dir + '/facial_emotion_recognition.h5') #loading the trained CNN model
classes = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [3]:
face_classifier = cv2.CascadeClassifier(current_dir + '/haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray,1.2, 4)
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 2)
        
        #cropping the frontal face from the image
        face_crop = gray[y:y+h, x:x+h]
        resized_face = cv2.resize(face_crop, (32,32), interpolation=cv2.INTER_AREA)
        face_array = img_to_array(resized_face).astype('float')/255.0
        face_array = np.expand_dims(face_array,axis=0)
        
        #predictions
        pred = model.predict(face_array)[0]
        pred_accuracy = str(round(pred.max() * 100, 2))
        pred_label = classes[pred.argmax()]
        
        cv2.putText(frame, pred_label, (x,y), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 3)
        cv2.putText(frame, 'accuracy: '+pred_accuracy, (0,0+50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 3)
            
         
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()